# $\alpha(k)$ likelihood components 

This notebook accompanies the paper **“Going beyond $S_8$: fast inference of the matter power spectrum from weak-lensing surveys”** and extracts components to write a likelihood based on the $\alpha(k)$ posteriors obtained in the main results. Please refer to the main Jupyter notebook for a step-by-step guide to reproduce the analysis. 

### 🔍 What this notebook does
Extracts the necessary components for an $\alpha(k)$ likelihood to run using `MontePython`. Specifically, from the $\alpha(k)$ posteriors, we want 
- $k$ vector 
- $\alpha(k)$ means 
- covariance martix in $\alpha(k)$ space 

Additionally, we also store 
- the effective redshift $z_{\rm eff}$ probed by the experiment or combination of experiments 
- the fiducial cosmology to compare to, stored as the nonlinear matter power spectrum $P(k, z_{\rm eff})$ at the $k$ vector above and the effective redshift $z_{\rm eff}$ 

These are then written to an `npz` output file that is read by `MontePython`. 

### 🛠 Requirements
This notebook uses:
- `numpy`
- `pyccl` for cosmological computations

### 📂 Folder structure
- `data/`: holds data products called by MP. Copy contents of directory into `montepython_public/data`
- `likelihoods/`: holds cls2pk MP likelihood. Copy contents of directory into `montepython_public/montepython/likelihoods`

In [1]:
import numpy as np
import pyccl

### $k,z$ values 

Per the paper, we use 24 logarithmic bins between $10^{-3}$ and $10^2{\rm Mpc}^{-1}$ and refer to Table 1 for the effective redshifts for each experiment. 

In [2]:
# logk bins
nk = 24
kk_edges = np.logspace(-3, 2, nk + 1)
kk = np.exp(0.5 * (np.log(kk_edges[1:]) + np.log(kk_edges[:-1])))

In [3]:
# effective z probed 
z_eff = 0.36

### Get fiducial cosmology at the specific $k,z$ 

In [4]:
# Planck 2018 LCDM cosmology with TT,TE,EE+lowE
planck_params = dict(omega_b=0.02236, omega_c=0.1202, ln10e10As=3.045, n_s=0.9649, H0=67.27)
params = dict(
    A_s=np.exp(planck_params['ln10e10As']) * 1e-10,
    Omega_c=planck_params['omega_c'] / (planck_params['H0'] / 100.)**2,
    Omega_b=planck_params['omega_b'] / (planck_params['H0'] / 100.)**2,
    h=planck_params['H0'] / 100.,
    n_s=planck_params['n_s'],
    m_nu=[0.06, 0., 0.],
    Neff=3.046)

# Cosmological model
cosmo = pyccl.Cosmology(**params)

In [5]:
# fiducial nonlinear matter power spectrum to compare to 
pk_nl_kz = pyccl.nonlin_matter_power(cosmo, kk, z_eff)

### Get $\alpha(k)$ means and covariance 

In [6]:
def load_chain(fname):
    return (np.vstack(np.load(fname)))

In [7]:
alpha_chain = load_chain('../chains/DES_nk24_sample_smoothness0.3_systTrue_ccovTrue.npy')

In [8]:
alpha_means = alpha_chain.mean(axis=0)
alpha_covmat = np.cov(alpha_chain, rowvar=False)

### Truncate all vectors and matrices to the constrained region 

This is done by hand based on reproducing Fig. 2 from the paper. For the specific example here, the truncation is `[7:-9]`

In [9]:
kk_trunc = kk[7:-9]
alpha_trunc = alpha_means[7:-9]
acov_trunc = alpha_covmat[7:-9, 7:-9]
pk_nl_kz_trunc = pk_nl_kz[7:-9]

Invert the covariance matrix to save time during likelihood computations 

In [10]:
a_icov_trunc = np.linalg.inv(acov_trunc)

### Save components

In [11]:
np.savez('./data/cls2pk/example', 
         kk=kk_trunc, z_eff=z_eff, 
         pk_fid=pk_nl_kz_trunc, 
         alpha=alpha_trunc, 
         aicov=a_icov_trunc)